# Exercises for the course "Declarative Problem Solving Paradigms in AI"

In [1]:
# Imports (some dependency errors may appear, you can ignore them for now)
# !pip install cpmpy numpy colorama networkx matplotlib requests z3-solver --quiet

import colorama
import cpmpy as cp
import numpy as np

## **Session 2: Reification, solving, debugging and explaining**

Welcome to the second exercise session of Declarative Problem Solving Paradigms in AI. In this session, we will cover a collection of concepts you would need to master as to efficiently and effectively create Constraint Programming models of your own:


1) Debugging

2) Solving

3) Explaining

4) Reification

We will cover each of them in a separate section; guiding you through an example, followed by a modelling exercise.

**Useful Resources:**
* CPMpy documentation: https://cpmpy.readthedocs.io/en/latest/index.html
* CPMpy quickstart: https://cpmpy.readthedocs.io/en/latest/modeling.html

### **Part 1: Debugging**

In this section, we'll look at a quite usefull topic for when designing your own Constraint Programming models: **debugging**.

You've made a model, and now you are trying to solve it. But oh no, you get:
- an error
- no solution
- an incorrect solution

We've gotten ourselves a BUG!



<!-- <div>
    <img src="attachment:e1a4c1fb-dfbc-43a1-a092-3244c1a23c06.png" width="300"/>
</div> -->


<div>
    <img src="https://pngpix.com/images/hd/cartoon-ladybug-illustration-7lugnuh6u9q5fm98.jpg" width="300"/>
</div>


<!-- ![image.png](attachment:e1a4c1fb-dfbc-43a1-a092-3244c1a23c06.png)![image.png](attachment:2b4e9a5b-1293-41a1-b1e2-3d549d0d301e.png) -->

From previous courses you should already have some experience with debugging. But now we are working with a declarative language, so how do you "*debug a model*"? There is no such thing as setting break-points and going through the lines of code step-by-step. The solver is assumed to be a "black box", inaccessible for us modelers.

Luckily, the expert modeler [Håkan Kjellerstrand](http://www.hakank.org/) has some tips for us:
- Test the model **early and often**
- When a model is not working, **activate the constraints one by one** to test which constraint (or combination) is the culprit.
- **Check the domains** of the decision variables

The idea is that by doing the first point often, there is a limited need for the second (which is more tedious).

The bug can be situated in one of three layers:
1. your model
2. the modeling library (CPMpy)
3. the solver

They are ordered from most likely to least likely! The last two are more meant for solver developers and for us, your teaching assistants (Tias' research group is responsible for the development of CPMpy, have a [look](https://github.com/CPMpy/cpmpy) at our work ;) )

We'll only cover the first type. We've seen some common modeling mistakes in the lectures. Let's see if you're able to spot all of them, and of couse provide a correction.

#### **0. Setup**

In [2]:
def used_capacity(x):
    """A small function to give you feedback whether you've correctly fixed the upcoming 'Warehouse Container Problem'"""
    a = np.sum(x * np.array([[2,3,5]]).T, axis=0)
    res = f"{'Over capacity: ' if any(a > 10) else 'OK: '}\n"
    for i,b in enumerate(a):
        res += f"{colorama.Fore.RED if b > 10 else colorama.Fore.GREEN}Container {i}: {b} / {10}\n"
    res += colorama.Style.RESET_ALL
    return res

#### **1. Syntax Errors**

The bugs to be found in the models below belong to the category of "**syntax errors**". These are of course the easiest to catch, since the Python interpreter will detect them for you and will raise an error.

##### **1.1. Worker Assignment**

A factory needs to assign 4 workers (W1, W2, W3, W4) to 4 different tasks (T1, T2, T3, T4). Each worker can perform one task at a time, and every task must be completed by exactly one worker.

Additionally:

- W1 can only work on Task 1 or Task 2.
- W2 cannot work on Task 3.
- W3 must work on either Task 3 or Task 4.
- W4 must work on a different task than Worker 2.

The goal is to assign each worker to a task under these constraints.

In [3]:
"""
TODO: Debug this model.

HINT: Carefully look at the error message you get.
"""

# Decision Variables
# - x[i] represents the task assigned to worker i (indexed from 1)
x = cp.intvar(1, 4, shape=4, name="x")

# Model
m = cp.Model()

# Constraints:
# - W1 can only work on Task 1 or Task 2
m += [x[0] == 1 | x[0] == 2]
# - W2 cannot work on Task 3
m += [x[1] != 3]
# - W3 must work on Task 3 or Task 4
m += [x[2] == 3 | x[2] == 4]
# - W4 must work on a different task than W2
m += [x[3] != x[1]]
# - All tasks must be assigned to different workers (all different constraint)
m += [cp.AllDifferent(x)]

# Solve the model
if m.solve():
    print("Solution found:")
    print("Task assignments:", x.value())
else:
    print("No solution found.")

TypeError: 1|x[0] is not valid because 1 is a number, did you forgot to put brackets? E.g. always write (x==2)|(y<5).

As seen in the lecture, be careful with the order of operations, also known as the strength with which the operators bind. `x == 1 & y == 2` will model `x == (1 & y) == 2` and not the intended `(x == 1) & (y == 2)`.

**Fixed**: first and third constraints:

In [4]:
"""
TODO: Debug this model.

HINT: Carefully look at the error message you get.
"""

# Decision Variables
# - x[i] represents the task assigned to worker i (indexed from 1)
x = cp.intvar(1, 4, shape=4, name="x")

# Model
m = cp.Model()

# Constraints:
# - W1 can only work on Task 1 or Task 2
m += [(x[0] == 1) | (x[0] == 2)]
# - W2 cannot work on Task 3
m += [x[1] != 3]
# - W3 must work on Task 3 or Task 4
m += [(x[2] == 3) | (x[2] == 4)]
# - W4 must work on a different task than W2
m += [x[3] != x[1]]
# - All tasks must be assigned to different workers (all different constraint)
m += [cp.AllDifferent(x)]

# Solve the model
if m.solve():
    print("Solution found:")
    print("Task assignments:", x.value())
else:
    print("No solution found.")

Solution found:
Task assignments: [2 4 3 1]


##### **1.2. Machine Configuration**

A manufacturing plant operates several machines that need to be calibrated regularly to ensure they are functioning at peak efficiency. The calibration involves adjusting four key machine settings (a, b, c, and d), each of which has a specific range of values.

The maintenance team has a calibration checklist:
- Setting a can be adjusted between 0 and 4.
- Setting b can be adjusted between 0 and 3.
- Setting c must be set between 1 and 4.
- Setting d must be set between 1 and 3.

Calibration Requirements:
- At most two settings can be set to exactly 3 to avoid overloading the machines with a high configuration in too many settings.
- At least one setting must be exactly 2 to ensure a certain degree of stability across the machine settings.

In [5]:
"""
TODO: Debug this model.
"""

# Decision Variables
# - Machine settings represented by integer variables
a = cp.intvar(0, 4, name="a")
b = cp.intvar(0, 3, name="b")
c = cp.intvar(1, 4, name="c")
d = cp.intvar(1, 3, name="d")

# - List of settings
e = [a, b, c, d]

# Model
m = cp.Model()

# Constraints
# - At most 2 settings can be exactly 3
m += cp.sum(e == 3) <= 2
# - At least 1 setting must be exactly 2
m += cp.sum(e == 2) >= 1

# Solve the model
if m.solve():
    print("Optimal machine settings found:")
    print("Setting a:", a.value())
    print("Setting b:", b.value())
    print("Setting c:", c.value())
    print("Setting d:", d.value())
else:
    print("No valid configuration found.")

TypeError: 'bool' object is not iterable

**Fixed**: use `cp.Count` instead:

In [6]:
"""
TODO: Debug this model.
"""

# Decision Variables
# - Machine settings represented by integer variables
a = cp.intvar(0, 4, name="a")
b = cp.intvar(0, 3, name="b")
c = cp.intvar(1, 4, name="c")
d = cp.intvar(1, 3, name="d")

# - List of settings
e = [a, b, c, d]

# Model
m = cp.Model()

# Constraints
# - At most 2 settings can be exactly 3
m += cp.Count(e, 3) <= 2
# - At least 1 setting must be exactly 2
m += cp.Count(e, 2) >= 1

# Solve the model
if m.solve():
    print("Optimal machine settings found:")
    print("Setting a:", a.value())
    print("Setting b:", b.value())
    print("Setting c:", c.value())
    print("Setting d:", d.value())
else:
    print("No valid configuration found.")

Optimal machine settings found:
Setting a: 2
Setting b: 0
Setting c: 1
Setting d: 1


#### **2. Unexpected UNSAT**

Now we'll look at more difficult bugs, the ones where the model does not have errors but still doesn't return a solution (when we were expecting one).

##### **2.1. Organizing the Harmony Festival**

The town of Meadowvale is buzzing with excitement as the annual Harmony Festival approaches. This year, a diverse group of friends and acquaintances has come together to make the festival a success. However, their relationships are complicated; some are good friends, while others have a history of conflict.

**Characters**
- **Sophie** : The enthusiastic organizer who is in charge of the **Food Stall**. She believes good food brings people together.
- **Jake** : A talented musician responsible for arranging **Live Music**. He loves performing but has a rivalry with another group: Emily.
- **Emily** : A passionate community member organizing a **Workshop** on sustainable living. She wants to educate attendees but is not on good terms with Jake due to past disagreements.
- **Oliver** : The festival's logistics manager. He aims to ensure everything runs smoothly but is caught in the middle of the tensions.

**Festival Events and Constraints**
- **Food Stall (A)**: If Sophie sets up a food stall, Jake must provide live music to create a lively atmosphere.
- **Live Music (B)**: Jake is excited to perform, but if he is on stage, Emily cannot conduct her workshop, as their past conflicts would create tension and distract the attendees.
- **Workshop (C)**: If Emily organizes her workshop, she insists that there must be a food stall, as refreshments will be crucial for participants. But she will not turn up of Jake is also there.

**Objective**
The goal is to maximize the number of events while avoiding conflicts.

In [7]:
"""
TODO: Debug this model.
"""

# Decision Variables
# - define binary variables for events
A = cp.boolvar(name="Food_Stall")  # True if there is a food stall
B = cp.boolvar(name="Live_Music")  # True if there is live music
C = cp.boolvar(name="Workshop")    # True if there is a workshop

# Model
model = cp.Model()

# Constraints
# 1) If there is a food stall, there must be live music
if A:
    model += B
# 2) If there is live music, there cannot be a workshop
if B:
    model += ~C
# 3) If there is a workshop, there must be a food stall, there can't be live music
if C:
    model += A
    model += ~B

model.maximize(A + B + C)

# Solve the model
if model.solve():
    print("Festival Setup:")
    print(f"Food Stall: {'Yes' if A.value() else 'No'}")
    print(f"Live Music: {'Yes' if B.value() else 'No'}")
    print(f"Workshop: {'Yes' if C.value() else 'No'}")
else:
    print("No feasible setup exists for the festival.")

No feasible setup exists for the festival.


**Fixed**: the if conditions weren't actually adding the constraints correctly, use `cp.implies` instead:

In [8]:
"""
TODO: Debug this model.
"""

# Decision Variables
# - define binary variables for events
A = cp.boolvar(name="Food_Stall")  # True if there is a food stall
B = cp.boolvar(name="Live_Music")  # True if there is live music
C = cp.boolvar(name="Workshop")    # True if there is a workshop

# Model
model = cp.Model()

# Constraints
# 1) If there is a food stall, there must be live music
model += A.implies(B)
# 2) If there is live music, there cannot be a workshop
model += B.implies(~C)
# 3) If there is a workshop, there must be a food stall, there can't be live music
model += C.implies(A & ~B)

model.maximize(A + B + C)

# Solve the model
if model.solve():
    print("Festival Setup:")
    print(f"Food Stall: {'Yes' if A.value() else 'No'}")
    print(f"Live Music: {'Yes' if B.value() else 'No'}")
    print(f"Workshop: {'Yes' if C.value() else 'No'}")
else:
    print("No feasible setup exists for the festival.")

Festival Setup:
Food Stall: Yes
Live Music: Yes
Workshop: No


##### **2.2. Treasure Hunt**

A group of treasure hunters is preparing for an expedition and needs to pack a limited-capacity knapsack for the journey. They have four valuable artifacts with different weights and values, and their challenge is to select at least three of them to maximize the total value they can carry without exceeding the knapsack's weight limit.

The hunters have identified four items with the following properties:

- Weights: [2, 3, 4, 5] (in kilograms)
- Values: [3, 4, 5, 6] (in monetary units)

The knapsack can hold a maximum weight of 10 kilograms, and the hunters must select a combination of artifacts that maximizes the total value while staying within the weight limit.

In [9]:
"""
TODO: Debug this model.

HINT: Keep the tips from Håkan Kjellerstrand in mind.
"""

# Parameters of the Problem
weights = [2, 3, 4, 5]  # weight of each item
values = [3, 4, 5, 6]   # value of each item
capacity = 10           # maximum weight capacity of the knapsack

n_items = len(weights)

# Decision Variables
x = cp.boolvar(shape=n_items) # a binary decision variable for each item (0 = don't take, 1 = take)
total_weight = cp.intvar(0, 10)
total_value = cp.intvar(0, 10)

# Model
model = cp.Model()

# Constraints
model += ( total_weight <= capacity )             # weight constraint
model += ( total_weight == cp.sum(x * weights) )  # total weight
model += ( total_value == cp.sum(x * values) )    # total value
model += ( cp.sum(x) >= 3 )                       # min number of items

# Objective
model.maximize(total_value) # objective to maximize the total value

# Solve the model
if model.solve():
    print("Optimal selection of items:", x.value())
    print("Total weight:", total_weight.value())
    print("Total value:", total_value.value())
else:
    print("No solution found")

No solution found


**Fixed**: the upper bound on the `total_value` was incorrect:

In [10]:
"""
TODO: Debug this model.

HINT: Keep the tips from Håkan Kjellerstrand in mind.
"""

# Parameters of the Problem
weights = [2, 3, 4, 5]  # weight of each item
values = [3, 4, 5, 6]   # value of each item
capacity = 10           # maximum weight capacity of the knapsack

n_items = len(weights)

# Decision Variables
x = cp.boolvar(shape=n_items) # a binary decision variable for each item (0 = don't take, 1 = take)
total_weight = cp.intvar(0, capacity)
total_value = cp.intvar(0, sum(values))

# Model
model = cp.Model()

# Constraints
model += ( total_value == cp.sum(x * values) )    # total value
model += ( total_weight == cp.sum(x * weights) )  # total weight
model += ( total_weight <= capacity )             # weight constraint
model += ( cp.sum(x) >= 2 )                       # min number of items

# Objective
model.maximize(total_value) # objective to maximize the total value

# Solve the model
if model.solve():
    print("Optimal selection of items:", x.value())
    print("Total weight:", total_weight.value())
    print("Total value:", total_value.value())
else:
    print("No solution found")

Optimal selection of items: [ True  True False  True]
Total weight: 10
Total value: 13


#### **3. Unexpected Solution**

Then there is the most difficult catergory of bugs, the ones where there is no error in the model, where the model does return a solution, but where the solution is not what we expected it to be. We've made a mistake when translating the problem description to the model formulation, and these mistakes can be very subtle. Have we forgotten a constraint, maybe an edge-case, does the constraint not express what we intended, etc?

##### **3.1. Warehouse Container Problem**

A warehouse needs to store different types of goods in containers. The warehouse has 4 containers, each with a limited capacity. There are 3 types of goods (A, B, and C) that need to be stored in these containers. The problem constraints are as follows:

- Each container can hold a maximum of 10 units of goods.
- Goods of type A require 2 units of space for each item.
- Goods of type B require 3 units of space for each item.
- Goods of type C require 5 units of space for each item.
- The warehouse needs to store exactly 5 units of goods of type A, 4 units of goods of type B, and 3 units of goods of type C.
- The total space used in each container must not exceed its capacity.

The objective is to assign the goods to the containers such that the capacity constraints are respected.

In [11]:
"""
TODO: Debug this model.
"""

# Decision Variables
# - x[i, j] represents how many units of good i are placed in container j
x = cp.intvar(0, 5, shape=(3, 4), name="x")  # 3 types of goods, 4 containers

# Model
model = cp.Model()

# Constraints:
# 1. Each container can hold a maximum of 10 units of goods
for j in range(4):
    model += [cp.sum(x[:, j]) <= 10]  # Total units of goods in container j
# 2. The warehouse must store exactly 5 units of good A, 4 units of good B, and 3 units of good C
model += [cp.sum(x[0, :]) == 5]  # 5 units of good A
model += [cp.sum(x[1, :]) == 4]  # 4 units of good B
model += [cp.sum(x[2, :]) == 3]  # 3 units of good C

# Solve the model
if model.solve():
    print("Solution found:")
    print(x.value())
    print(used_capacity(x.value()))
else:
    print("No solution found.")

Solution found:
[[0 0 0 5]
 [0 0 0 4]
 [0 0 2 1]]
Over capacity: 
Container 0: 0 / 10
Container 1: 0 / 10
Container 2: 10 / 10
Container 3: 27 / 10



We forgot to take the 'units of space' for each of the items into account when calculating the used capacity.

In [12]:
"""
TODO: Debug this model.
"""

# Parameters of the Problem
# - space requirements for each type of good: A takes 2 units, B takes 3 units, C takes 5 units
space_requirements = [2, 3, 5]

# Decision Variables
# - x[i, j] represents how many units of good i are placed in container j
x = cp.intvar(0, 5, shape=(3, 4), name="x")  # 3 types of goods, 4 containers

# Model
model = cp.Model()

# Constraints:
# 1. Each container can hold a maximum of 10 units of goods, accounting for space requirements
for j in range(4):
    model += cp.sum(x[:, j] * space_requirements) <= 10
# 2. The warehouse must store exactly 5 units of good A, 4 units of good B, and 3 units of good C
model += [cp.sum(x[0, :]) == 5]  # 5 units of good A
model += [cp.sum(x[1, :]) == 4]  # 4 units of good B
model += [cp.sum(x[2, :]) == 3]  # 3 units of good C

# Solve the model
if model.solve():
    print("Solution found:")
    print(x.value())
    print(used_capacity(x.value()))
else:
    print("No solution found.")

Solution found:
[[0 1 0 4]
 [0 1 3 0]
 [2 1 0 0]]
OK: 
Container 0: 10 / 10
Container 1: 10 / 10
Container 2: 9 / 10
Container 3: 8 / 10

